In [10]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html
from dash import dash_table
from threading import Timer
import webbrowser

In [11]:
# This class of functions returns Dataframe version of reports available in Bloomberg AIM

class aim_manipulation:

    # This function returns a dataframe for the cash ladder by cleaning the ERA report from unnecessary rows and columns
    # Please note to change "iloc[:107]" if funds are added or removed
    
    def cash_ladder(path, reformat):
    
        df = pd.read_excel(path, skiprows = 6).drop(columns = ["Unnamed: 1"]).rename(columns = {"Settle Date Cash Amount" : "Account", 
                                                                                            
                                                                                            "Unnamed: 2" : "Currency"}).iloc[:109].fillna(method = "ffill")            
        def format_with_commas(value):
            if isinstance(value, (int, float)):
        
                return f'{value:,.0f}'
        
            else:
                                    
                return value
            
        if reformat == True:
            
            return df.applymap(format_with_commas)
        
        else:
            
            return df      
    
    #This function returns a dataframe for the financing trade blotter by cleaning the XFIN export file
    
    def financing_trade_blotter(path):
    
        df = pd.read_excel(path)
    
        for col in df:
        
            if df[col].dtype == "object":
            
                df[col] = df[col].str.lstrip()
            
                df[col] = df[col].str.rstrip()
        
        df = df.drop(0, axis = 0)

        return df
    
    def unwind_blotter(path, account, curr, tran_type, term_dt):
        
        # This function manipulates the financing trade blotter by removing unnecessary columns and filtering by transaction
        # type and currency
        # Accounts are COOLABAH funds' short names, such as LSCF, RAERICH... and must be entered as a list. If all accounts 
        # need to be selected input "all_account".
        # Currencies are AUD, EUR, GBP, USD and must be entered as a list.
        # Transaction types are RR, CRR, DO, RP
        # - RR: Reverse Repo (We lend the bond), used for funding
        # - CRR: Reverse Repo (We lend the bond), used for funding
        # - DO: Deposit
        # - RP: Repo (We borrow the bond), use to short bonds
        
        df = aim_manipulation.financing_trade_blotter(path)
        
        if account != "all_account":
            
            df = df[df["Account"].isin(account)]
        
        df = df[["Account", "Tran Type", "Broker", "Collateral", "Identifier", "Quantity", "Curr", "Money", "Rate", "SetDt"]]
        
        df["SetDt"] = pd.to_datetime(df["SetDt"], format = "%m/%d/%y")
               
        df["TermDt"] = pd.to_datetime(datetime.strptime(term_dt, '%d/%m/%Y'))
        
        df = df[(df["Curr"].isin(curr)) & (df["Tran Type"].isin(tran_type))].reset_index(drop = True)
        
        df["DayCount"] = df["Curr"].map(day_count)
        
        df["Daily Interest"] = df["Money"] * ((df["Rate"] / 365) / 100)

        df["Accr Interest"] = df["Money"] * (df["Rate"] * ((df["TermDt"] - df["SetDt"]).dt.days / 365) / 100)
        
        df["Term Amount"] = df["Money"] + df["Accr Interest"]
                
        return df
    
    def collateral_positions(path):
        
        df = pd.read_excel(path, skiprows = [1])
        
        df.rename(columns = {
            
            df.columns[8] : "T+0",
            
            df.columns[9] : "T+1",
            
            df.columns[10] : "T+2",
            
            df.columns[11] : "T+3",
            
            df.columns[12] : "T+4",
            
        },
            
            inplace = True
            
        )
        
        return df

In [12]:
# This class of functions solves different cases of Knapsack problems using dynamic programming algorithms.

class knapsack:
    
    # This function solves a 0-1 Knapsack problem using dynamic programming through a recursion algorithm.
    # The output is a string containing the indices of the optimal items to include in the Knapsack
    
    #Inputs:
    # v: Values, index 0 is a dummy
    # W: Weights, index 0 is a dummy
    # n: Number of distinct items
    # W:  Knapsack capacity
    
    def knapsack_01(v, w, n, W):
        
    # Initialize a (n + 1) x (W + 1) matrix where each element is set to -1
    # This will be used to store the maximum value that can be achieved with the first items and a weight limit j.
    
        value = [[-1 for _ in range(W + 1)] for _ in range(n + 1)]
    
    # Function m is defined to compute the maximum value that can be achieved using the first i items and a weight limit j.
    
        def m(i, j):
        
    # If i is 0 (no items) or j is 0 or less (no weight capacity), the value is 0.
        
            if i == 0 or j <= 0:
            
                value[i][j] = 0
            
                return
    
    # If value[i-1, j] is -1 it has not been calculated yet, so we call m(i-1, j) to ensure the value is computed.
    
            if value[i - 1][j] == -1:
            
                m(i - 1, j)
   
    # If the weight of the item i is greater than j, item i cannot be included in the Knapsack. 
    # Thus, value[i][j] is set to value[i-1][j] (the value without including item i).

            if w[i] > j:
            
                value[i][j] = value[i - 1][j]
    
    # If item i can fit the bag, the function first ensures that value[i-1][j-w[i]] has been calculated (== -1)
    # The, value[i][j] is set to the maximum of not including item "i" (value[i-1][j]) and including item "i"
    # (value[i-1][j-w[i]] + v[i])
    
            else:
            
                if value[i - 1][j - w[i]] == -1:  # m(i - 1, j - w[i]) has not been calculated, we have to call function m
                
                    m(i - 1, j - w[i])
                
                value[i][j] = max(value[i - 1][j], value[i - 1][j - w[i]] + v[i])

    # Run m(n, W)
    
        m(n, W)
    
        indices = []
 
        def find_indices(i, j):
        
            if i == 0:
            
                return
        
            if value[i][j] > value[i-1][j]:
            
                indices.append(i)
            
                find_indices(i-1, j-w[i])
            
            else:
            
                find_indices(i-1, j)
    
        find_indices(n, W)
    
        return indices

In [13]:
# This class of functions returns dashboards to clearly display data in tables.

class dashboards:
    
    def simple_dashboard(dataframe, title, port, size):
    
    # Initialise the Dash app
    
        app = Dash(__name__)

    # Nested function to create a DataTable component from a DataFrame.

        def create_data_table(df, index):
        
            return html.Div([
                    
                html.H2(f"Table {index + 1}"),
                
                dash_table.DataTable(
                    
                    id = f'data-table-{index}',
                    
                    columns = [{"name": i, "id": i} for i in df.columns],
                    
                    data = df.to_dict('records'),
                    
                    page_size = size,
                    
                    filter_action = "native",
                    
                    sort_action = "native",
                    
                    sort_mode = "multi",
                    
                )
                
            ])

    # Generate layout with multiple DataTables

        app.layout = html.Div(
            
            [html.H1(title)] +
            
            [create_data_table(df, i) for i, df in enumerate(dataframe)]
            
        )

        # Function to open the browser after server starts
        
        def open_browser():
            
            webbrowser.open_new("http://127.0.0.1:" + port + "/")

        if __name__ == '__main__':
            
            # Start the server
            
            Timer(1, open_browser).start()
            
            app.run_server(port = port)          
            

In [14]:
# This class of functions help with opening, downsizing and closing repos

class repo_trading:
    
    def close_repo(collateral, financing):
                      
        def first_short_position(collateral):
            
# Create a matrix with the collateral postions for a 5-day horizon and find the first day a balance becomes negative 
# for each security
            
            df_matrix = collateral[["T+0", "T+1", "T+2", "T+3", "T+4"]]
            
            negative_indices = []
            
            for index, row in df_matrix.iterrows():
                
                for col in df_matrix.columns:
                    
                    if row[col] < 0:
                        
                        negative_indices.append((index, col))
                        
                        break
                        
            return negative_indices
        
        short_positions = first_short_position(collateral) 
        
        df_short = collateral.loc[[pos[0] for pos in short_positions], # extract index from the rows that have a negative balance
                                
                                ["Account", "Curr", "Security", "ISIN"]].assign( # specify columns to select
            
            Position = [collateral.loc[pos[0], pos[1]] for pos in short_positions], 
            
            Settlement = [pos[1] for pos in short_positions]).sort_values(by = ["Account", "Security"],
                                                                         
                                                                         ascending = [True, True]).reset_index(drop = True)
        
        instructions = []
        
        for i, short in df_short.iterrows():
            
            df_temp = financing[(financing["Identifier"] == short["ISIN"]) & 
                               
                               (financing["Account"] == short["Account"])].sort_values(by = ["Rate", "Quantity"], 
                                                                                          
                                                                                          ascending = [False, True]).reset_index(drop = True)
            
            position_to_cover = short["Position"]
            
            for j, repo in df_temp.iterrows():
                
                if position_to_cover > 0:
                    
                    break
                    
                if  position_to_cover + repo["Quantity"] > 0:
                
                    to_reopen = position_to_cover + repo["Quantity"]
                    
                else:
                    
                    to_reopen = 0
                    
                instructions.append({
                    
                    "Account" : repo["Account"],
                    
                    "Collateral" : repo["Collateral"],
                    
                    "Identifier" : repo["Identifier"],
                    
                    "Close" : repo["Quantity"],
                    
                    "Open" : to_reopen,
                    
                    "Currency" : repo["Curr"],
                    
                    "Settlement": short["Settlement"],
                    
                    "Rate" : repo["Rate"],
                    
                    "Broker" : repo["Broker"]                    
                    
                })
                
                position_to_cover += repo["Quantity"]
                
        df_instructions = pd.DataFrame(instructions)
            
        return df_instructions